In [60]:
import numpy as np
import import_ipynb
import sinusfitnessfunction as ff

In [61]:
class Genetic_Algorithm:
    def __init__(
        self,
        n_genes,
        n_alleles,
        size_population,
    ):
        self.n_genes = n_genes
        self.n_alleles = n_alleles
        self.total_alleles = self.n_alleles * self.n_genes
        self.size_population = size_population
        self.chromosomes = []
        self.fitness = []
    
    def init_population(self):
        self.chromosomes = np.array([[1 if np.random.rand() > 0.5 else 0 for j in range(self.total_alleles)]for i in range(self.size_population) ])


In [62]:
Genetic_Algorithm1 = Genetic_Algorithm(
    3, 3, 5
)
Genetic_Algorithm1.chromosomes

[]

In [63]:
Genetic_Algorithm1.init_population()
Genetic_Algorithm1.chromosomes

array([[1, 0, 1, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 1, 1, 1, 0, 0, 1, 0],
       [1, 1, 1, 1, 0, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0, 0]])

In [64]:
chromosome = Genetic_Algorithm1.chromosomes[0][::-1]
n_alleles = 3
n_genes = 3
conversion = []
for i in range(n_genes):
    decimal_conversion = []
    for j in range(i*n_alleles, (i+1)*n_alleles):
        decimal_conversion.append(chromosome[j]*2**(j-n_alleles*i))
    conversion.append(np.sum(decimal_conversion))
print(conversion)


[3, 5, 5]
